# Change the ecoinvent version of the mapping file

In [1]:
import pandas as pd
import bw2data as bd
from mescal import *
import itertools

In [2]:
esm_location = 'CH'
ei_version_from = '3.8'
ei_version_to = '3.10'

In [3]:
bd.projects.set_current(f"ecoinvent{ei_version_to}")

In [4]:
name_comp_db = f'ecoinvent_cutoff_{ei_version_from}_remind_SSP2-Base_2020_with_CPC_comp_{esm_location}'

In [5]:
mapping = pd.read_csv(f"energyscope_data/{esm_location}/mapping_{ei_version_from}_linked.csv")

In [6]:
updated_mapping, unit_changes = change_ecoinvent_version_mapping(mapping, ei_version_from, ei_version_to, name_comp_db)

BATTERY Construction
Old: battery, Li-ion, rechargeable, prismatic - market for battery, Li-ion, rechargeable, prismatic - GLO
New: battery, Li-ion, LiMn2O4, rechargeable, prismatic - market for battery, Li-ion, LiMn2O4, rechargeable, prismatic - GLO
CEMENT_PROD Operation
Old: cement, alternative constituents 21-35% - cement production, alternative constituents 21-35% - CH
New: cement, CEM II/B - cement production, CEM II/B - CH
CEMENT_PROD_HP Operation
Old: cement, alternative constituents 21-35% - cement production, alternative constituents 21-35% - CH
New: cement, CEM II/B - cement production, CEM II/B - CH
CO2_METHANOL Operation
Old: methanol - methanol production - GLO
New: methanol - methanol production, natural gas reforming - CN
CUMENE_PROCESS Operation
Old: cumene - cumene production - RER
New: cumene - cumene production, benzene alkylation - RER
ELEC_STO Construction
Old: battery, Li-ion, rechargeable, prismatic - market for battery, Li-ion, rechargeable, prismatic - GLO
New:

In [7]:
updated_mapping.head()

,Name,Type,Product,Activity,Location,Database
0,ALKALINE_ELECTROLYSIS,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.10_remind_SSP2-Base_2020_wi...
1,ALKALINE_ELECTROLYSIS_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.10_remind_SSP2-Base_2020_wi...
2,ALKALINE_ELECTROLYSIS_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.10_remind_SSP2-Base_2020_wi...
3,ALKALINE_ELECTROLYSIS_STACK,Construction,"electrolyzer, 1MWe, AEC, Stack","electrolyzer production, 1MWe, AEC, Stack",RER,ecoinvent_cutoff_3.10_remind_SSP2-Base_2020_wi...
4,ALKALINE_ELECTROLYSIS_STACK_DECOM,Construction,"used fuel cell stack, 1MWe, AEC","treatment of fuel cell stack, 1MWe, AEC",RER,ecoinvent_cutoff_3.10_remind_SSP2-Base_2020_wi...


In [8]:
base_db = load_multiple_databases(list(updated_mapping.Database.unique()))
unlinked = test_mapping_file(updated_mapping, base_db)

Getting activity data


100%|██████████| 34093/34093 [00:00<00:00, 52303.27it/s]


Adding exchange data to activities


100%|██████████| 1154525/1154525 [01:05<00:00, 17592.76it/s]


Filling out exchange data


100%|██████████| 34093/34093 [00:05<00:00, 6563.86it/s] 


Mapping successfully linked to the database


In [9]:
if len(unlinked) == 0:
    updated_mapping.to_csv(f"energyscope_data/{esm_location}/mapping_{ei_version_to}.csv", index=False)
else:
    print(f"Unlinked exchanges: {unlinked}")

# Adapt the unit conversion file corresponding to the new mapping file

In [10]:
unit_conversion = pd.read_csv(f"energyscope_data/{esm_location}/unit_conversion_{ei_version_from}.csv")

In [11]:
unit_changes

[]

In [12]:
# Add new unit conversion factors
new_unit_conversion_factors = {
    ('OCGT_LARGE', 'Operation'): 3.6 * 2.5,  # 3.6 MJ of heat per kWh, 2.5 kWh NG per kWh electricity
    ('OCGT_SMALL', 'Operation'): 3.6 * 2.5,  # 3.6 MJ of heat per kWh, 2.5 kWh NG per kWh electricity
}

In [13]:
unit_conversion = update_unit_conversion_file(unit_conversion, unit_changes, new_unit_conversion_factors)

In [14]:
unit_conversion.sort_values(by=['Name', 'Type']).to_csv(f"energyscope_data/{esm_location}/unit_conversion_{ei_version_to}.csv", index=False)